**Modelos Ingeniería Logística**

1.- *3 Modelos del TSP utilizando las variables u, j o f*

1.1.- Primer modelo (Usando la variable *u* para conocer la posición que ocupa cada nodo en la ruta final)

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
EPS = 0.001

def TSP1():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
    u = {}
    for i in otros:
        u[i] = solver.IntVar(0.0, solver.infinity(), 'u[%i]' % i)  

    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  u[j] >= u[i] + x[i,j] - (n-2)*(1-x[i,j]) + (n-3)*x[j,i]   ) for i in otros for j in otros if j!=i ]
        
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP1()


1.2.- Segundo modelo (Usando la variable *v* para llevar el control de que es nodo es visitado antes que otro)